In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
from pydrake.all import StartMeshcat

# if you're not launching jupyter from the repo root, fix the path:
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

meshcat = StartMeshcat()
print("Meshcat:", meshcat.web_url())

In [ ]:
# simulate_two.py
from rocky.build import build_robot_diagram_two
from rocky import utils
import os
import numpy as np

# assume repo_root and meshcat are already defined in your notebook / script
urdf = os.path.join(repo_root, "rocky.urdf")   # version WITHOUT wall_to_world

bundle = build_robot_diagram_two(
    urdf_path=urdf,
    time_step=1e-3,
    gravity_vec=(0., 0., -9.81),
    meshcat=meshcat,
    enemy_target_xyz=(0., 0.0, 1.1),      # 10 cm off the wall, slightly higher
    enemy_target_rpy=(0.0, 0.0, 0.0),        # facing straight out
    enemy_target_radius=0.05,                # 3 cm sphere
    enemy_target_rgba=(0.2, 0.9, 0.2, 1.0),  # green target
)


plant, ctx, sim = bundle.plant, bundle.plant_context, bundle.simulator
ally, enemy = bundle.ally, bundle.enemy


shoulder_angle = np.deg2rad(-45)
elbow_angle = np.deg2rad(-60)


# Pose each arm independently
utils.set_angles_for_instance(plant, ctx, ally,  shoulder_angle, elbow_angle)
utils.set_angles_for_instance(plant, ctx, enemy, 0.0, 0.0)

# Passive sim for both
apply_control = utils.make_control_applier(plant, ally)
apply_control(ctx, np.ones(2))

sim.AdvanceTo(sim.get_context().get_time() + 1.0)

In [ ]:
# Get frames
A_EE = plant.GetFrameByName("glove", model_instance=ally)
T    = plant.GetFrameByName("enemy_target", model_instance=enemy)

# World poses
X_W_ee = plant.CalcRelativeTransform(ctx, plant.world_frame(), A_EE)
X_W_t  = plant.CalcRelativeTransform(ctx, plant.world_frame(), T)

# Relative pose from ally EE to target (use for control)
X_ee_t = plant.CalcRelativeTransform(ctx, A_EE, T)

# For just the target point in world:
p_W_t = X_W_t.translation()

print("A: ", X_ee_t)
print("T: ", p_W_t)

In [ ]:

# iLQR MPC step example
import numpy as np
from rocky import ilqr
from pydrake.all import Rgba, RigidTransform
from pydrake.geometry import Sphere



enemy_target_frame = plant.GetFrameByName('enemy_target', model_instance=enemy)
enemy_target_location = plant.CalcRelativeTransform(ctx, plant.world_frame(), enemy_target_frame)
ally_wall_frame = plant.GetFrameByName("wall", model_instance=ally)
ally_wall_location = plant.CalcRelativeTransform(ctx, plant.world_frame(), ally_wall_frame) 
target_pos = np.array([0.5, 0.3])

cost_params = ilqr.CostParams(
    Q= np.diag([2.0, 5.0, 10, 10]),
    R= 10 * np.eye(2),
    wall_position= ally_wall_location.translation()[0],
    avoid_sigma=0.2,
    avoid_weight=150
)
ctrl = ilqr.ILQRController(horizon=100, dt=0.05, params=ilqr.ArmParams(), cost=cost_params, verbose=True)


print('----------------------------------------------')
print('Ally Location:', plant.GetPositions(ctx, model_instance=ally))
print('Ally Velocities:', plant.GetVelocities(ctx, model_instance=ally))
print('Enemy Target Location:', enemy_target_location.translation())
print('Wall Position:', cost_params.wall_position)
print(ctrl)
print('----------------------------------------------')





use_defined_target = True
u_traj_prev = None
control_hz = 1.0 / ctrl.dt

In [ ]:
for _ in range(1000):
    # read state from Drake
    q = plant.GetPositions(ctx, model_instance=ally)
    dq = plant.GetVelocities(ctx, model_instance=ally)
    x = np.hstack([q, dq])

    # world target/enemy positions -> planar (x,z) used by ilqr
    X_W_target = plant.CalcRelativeTransform(ctx, plant.world_frame(), enemy_target_frame)
    if not use_defined_target:
        target_pos = np.array([X_W_target.translation()[0], X_W_target.translation()[2]])

    # enemy glove position (avoid); here we mirror the target frame
    enemy_pos = target_pos.copy()

    # Visual marker for goal
    meshcat.SetObject("simple_marker", Sphere(0.07))
    meshcat.SetTransform("simple_marker",
                        RigidTransform([target_pos[0], 0, target_pos[1]]))

    x_next, result = ctrl.step_mpc(x, target_pos=target_pos, enemy_pos=enemy_pos, prev_u_trj=u_traj_prev)
    u_traj_prev = result.u_trj

    # apply first control to Drake
    apply_control(ctx, result.u_trj[0])

    # advance physics by one control period
    sim.AdvanceTo(sim.get_context().get_time() + ctrl.dt)

print('Last state (pred):', x_next)
